In [30]:
import pandas as pd
import os, fnmatch
import xlrd
import difflib
import re
             

In [31]:
# Importing Phase 2 lB data into pandas data frame
df_dac = pd.read_sas(r'\\cdsesub1\\evsprod\\BLA761029\\0014\\m5\\datasets\\205ms201\\tabulations\\sdtm\\lb.xpt')

In [32]:
data_elements_file =  pd.ExcelFile("C:\Users\Md.Shamsuzzaman\Documents\OCS\AIH\data_elements_ver2.xlsx")

In [33]:
# Load the xls file's Sheet1 as a dataframe
df = data_elements_file.parse('LB')
#df.head()

In [34]:
# not collecting all the var at this moment - like ACTARM
df = df.iloc[:25,:]
df.tail(3)

,Variable Name,Variable Label,Type,"Controlled Terms, Codelist or Format",Role,CDISC Notes,Core,Remark
22,LBDTC,Date/Time of Specimen\nCollection,Char,ISO 8601,Timing,NaN,Exp,NaN
23,LBENDTC,End Date/Time of\nSpecimen Collection,Char,ISO 8601,Timing,NaN,Perm,NaN
24,LBDY,Study Day of Specimen\nCollection,Num,NaN,Timing,"1. Study day of specimen collection, measured ...",Perm,NaN


In [35]:
# Data elements needs to be extracted from LB domains
LB_data_elements = list(df['Variable Name'])

In [36]:
# Not all are present in Daclizumab LB domain. Hence, create an empty df with variables needed first

lb_df = pd.DataFrame(index = range(len(df_dac)), columns = LB_data_elements)


In [37]:
# find common col var 
common_var = list(set(LB_data_elements).intersection(list(df_dac)))

In [38]:
# find the vars which are present in lb_df and missing in df_dac
list(set(LB_data_elements) - set(common_var))

[u'APPLICID', u'LBTOX', u'LBENDTC', u'LBFAST', u'LBSTNRC']

In [39]:
# find the vars which are present in df_dac and missing in lb_df
list(set(list(df_dac)) - set(common_var))

[u'LBORRES',
 u'LBORRESU',
 u'LBORNRHI',
 u'LBSCAT',
 u'EPOCH',
 u'LBREFID',
 u'LBNAM',
 u'LBORNRLO',
 u'LBNRIND']

In [40]:
# fill up the dm_df with daclizumab data
lb_df['APPLICID'] = 'BLA761029'
lb_df[common_var] = df_dac[common_var]

In [41]:
lb_df.head()

,APPLICID,STUDYID,DOMAIN,USUBJID,LBSEQ,LBTESTCD,LBTEST,LBCAT,LBSTRESC,LBSTRESN,...,LBREASND,LBBLFL,LBFAST,LBTOX,LBTOXGR,VISITNUM,VISIT,LBDTC,LBENDTC,LBDY
0,BLA761029,205MS201,LB,205MS201/100-001,1.0,ALP,Alkaline Phosphatase,BLOOD CHEMISTRY,62,62.0,...,,,NaN,NaN,0,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0
1,BLA761029,205MS201,LB,205MS201/100-001,2.0,ALT,Alanine Aminotransferase,BLOOD CHEMISTRY,18,18.0,...,,,NaN,NaN,0,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0
2,BLA761029,205MS201,LB,205MS201/100-001,3.0,AST,Aspartate Aminotransferase,BLOOD CHEMISTRY,14,14.0,...,,,NaN,NaN,0,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0
3,BLA761029,205MS201,LB,205MS201/100-001,4.0,BICARB,Bicarbonate,BLOOD CHEMISTRY,25.1,25.1,...,,,NaN,NaN,,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0
4,BLA761029,205MS201,LB,205MS201/100-001,5.0,BUN,Blood Urea Nitrogen,BLOOD CHEMISTRY,3,3.0,...,,,NaN,NaN,,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0


In [42]:
# Check the LBTEST code values
dac_cd = set(list(lb_df['LBTESTCD']))

In [43]:
# Load the xls file's Sheet1 as a dataframe
df_code = data_elements_file.parse('LB CodeList')
df_code = df_code.iloc[:125,:]
df_code.tail(5)

,LBTESTCD,LBTEST,LBTESTDSC,LBCAT,LBSTRESU
120,HCVGTYPE,HCV Genotype,NaN,MICROBIOLOGY,NaN
121,HCRTQE,HCV RNA Taq 2.0-Exp,NaN,HEMATOLOGY,NaN
122,HEVRNA,Hepatitis E Virus RNA,NaN,HEMATOLOGY,NaN
123,HIGMC,Herpes Sim.Virus 1/2 IgM IFA Confirm,NaN,SPECIAL CHEMISTRY,NaN
124,HIGMIN,Herpes Sim.Virus 1/2 IgM Interpretation,NaN,MICROBIOLOGY,NaN


In [44]:
# code elements needs to be extracted from LB domains
req_code = list(df_code['LBTESTCD'])

In [45]:
# find common col var 
common_var = list(set(req_code).intersection(list(dac_cd)))

In [46]:
# find the code which are present in lb_df and missing in dac_cd
unique_req_cd = list(set(req_code) - set(common_var))

In [47]:
# find the code which are unique to dac_code
unique_dac_cd = list(set(dac_cd) - set(common_var))

In [48]:
df_dac_unique = df_dac.drop_duplicates(['LBTESTCD'], keep = 'first')

In [49]:
# req_code df with unique req code 
unique_df_code = df_code[df_code.LBTESTCD.isin(unique_req_cd)]

In [50]:
#dac_df with unique dac_code 
df_dac_unique = df_dac_unique[df_dac_unique.LBTESTCD.isin(unique_dac_cd)]

In [51]:
# dac_df dictionary with unique LBTEST and LBTESTCD
unique_dac_lbtest = dict(zip(df_dac_unique.LBTESTCD,df_dac_unique.LBTEST))

In [52]:
# req_code_df dictionary with unique LBTEST and LBTESTCD
unique_req_lbtest = dict(zip(unique_df_code.LBTESTCD,unique_df_code.LBTEST))

In [53]:
for k,v in unique_req_lbtest.items():
    for k1,v1 in unique_dac_lbtest.items():
        if difflib.SequenceMatcher(None,v,v1).ratio() >=0.7 :
            print v,'\t\t',v1,'\t\t',k,'\t\t',k1

Urea Nitrogen 		Blood Urea Nitrogen 		UREAN 		BUN
Triacylglycerol Lipase (U/L) 		Triacylglycerol Lipase 		LIPASET 		LIPASE
Lymphocytes Activated  		Lymphocytes Atypical 		LYMA 		LYMAT
Eosinophils/Total Cells 		Eosinophils/Leukocytes 		EOSCE 		EOSLE
Lymphocytes/Total Cells 		Lymphocytes/Leukocytes 		LYMCE 		LYMLE


In [54]:
# Dict to replace the LBTESTCD with
matched_LBTESTCD = {'Lipase' : 'LIPASET', 'UREAN':'BUN'}

In [78]:
#lb_df.ix[lb_df.LBTESTCD == 'BUN', 'LBTESTCD'] = 'UREAN'

In [55]:
# Replace non SDTM LBTSTCDs with SDTM LBTSTCDs
for k, v in matched_LBTESTCD.items():
    lb_df.ix[lb_df.LBTESTCD == v, 'LBTESTCD'] = k

In [27]:
# drop rows which are not present in req_code
lb_df = lb_df[lb_df['LBTESTCD'].isin(req_code)]

In [28]:
lb_df.to_csv("C:/Users/Md.Shamsuzzaman/Documents/OCS/AIH/Daclizumab/205MS201_lb-5.csv",index = False)

In [89]:
lb_df.head()

,APPLICID,STUDYID,DOMAIN,USUBJID,LBSEQ,LBTESTCD,LBTEST,LBCAT,LBSTRESC,LBSTRESN,...,LBREASND,LBBLFL,LBFAST,LBTOX,LBTOXGR,VISITNUM,VISIT,LBDTC,LBENDTC,LBDY
0,BLA761029,205MS201,LB,205MS201/100-001,1.0,ALP,Alkaline Phosphatase,BLOOD CHEMISTRY,62,62.0,...,,,NaN,NaN,0,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0
1,BLA761029,205MS201,LB,205MS201/100-001,2.0,ALT,Alanine Aminotransferase,BLOOD CHEMISTRY,18,18.0,...,,,NaN,NaN,0,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0
2,BLA761029,205MS201,LB,205MS201/100-001,3.0,AST,Aspartate Aminotransferase,BLOOD CHEMISTRY,14,14.0,...,,,NaN,NaN,0,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0
3,BLA761029,205MS201,LB,205MS201/100-001,4.0,BICARB,Bicarbonate,BLOOD CHEMISTRY,25.1,25.1,...,,,NaN,NaN,,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0
4,BLA761029,205MS201,LB,205MS201/100-001,5.0,UREAN,Blood Urea Nitrogen,BLOOD CHEMISTRY,3,3.0,...,,,NaN,NaN,,10.0,SCREENING,2010-02-05T10:30,NaN,-20.0


In [56]:
lb_df.LBTESTCD.nunique()

99